## Creation of Panel data Set
## Stockout generation for hourly sales

get the original hourly sales, add 0 sales for all products, attach the stockout of the chosen X product
create the complementary dummy from the co-occurrence
finalize the panel data

In [1]:
import pandas as pd
from pytz import timezone
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import ast
import re
from dateutil import parser
from datetime import datetime, timedelta
import numpy as np
import statsmodels.formula.api as smf
from tabulate import tabulate
import pickle

In [2]:
hourly_sales = pd.read_csv("hourly_sales.csv")

In [3]:
hourly_sales = hourly_sales[['product_id','year','month','day','hour','day_of_week','hourly_sales','instock_fraction']]
hourly_sales

,product_id,year,month,day,hour,day_of_week,hourly_sales,instock_fraction
0,0,2022,1,1,10,7,1,1.0
1,0,2022,1,7,22,6,2,1.0
2,0,2022,1,8,10,7,1,1.0
3,0,2022,1,8,12,7,1,1.0
4,0,2022,1,8,13,7,1,1.0
...,...,...,...,...,...,...,...,...
732767,999,2023,3,17,7,6,1,1.0
732768,999,2023,3,17,9,6,1,1.0
732769,999,2023,3,21,1,3,1,1.0
732770,999,2023,3,21,19,3,1,1.0


In [4]:
stockout_all_products = pd.read_csv("stock_out_all_products.csv")

/var/folders/95/8jfgx8yx3sn4c62z5ksq2w840000gn/T/ipykernel_31369/2098527068.py:1: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191) have mixed types. Specify dtype option on import or set low_memory=False.
  stockout_all_products = pd.read_csv("stock_out_all_products.csv")


In [5]:
stockout_all_products = stockout_all_products[['product_id','stock_out_timestamps']]

In [6]:
hourly_sales_sample = hourly_sales.copy()

In [7]:
start_date = '2022-01-01'
end_date = '2023-01-01'

# Create a range of hourly timestamps
timestamps = pd.date_range(start=start_date, end=end_date, freq='H')

# Get unique product IDs from the hourly_sales_sample dataframe
product_ids = hourly_sales_sample['product_id'].unique()

# Create a dataframe with all possible combinations of product_id and hour
all_combinations = pd.MultiIndex.from_product([product_ids, timestamps], names=['product_id', 'timestamp'])
all_combinations = pd.DataFrame(index=all_combinations).reset_index()

# Separate the timestamp column into year, month, day, and hour
all_combinations['year'] = all_combinations['timestamp'].dt.year
all_combinations['month'] = all_combinations['timestamp'].dt.month
all_combinations['day'] = all_combinations['timestamp'].dt.day
all_combinations['hour'] = all_combinations['timestamp'].dt.hour

all_combinations = all_combinations.drop(all_combinations[all_combinations['timestamp'] == '2023-01-01 00:00:00'].index)

# Reset the index of the dataframe
all_combinations = all_combinations.reset_index(drop=True)

all_combinations

,product_id,timestamp,year,month,day,hour
0,0,2022-01-01 00:00:00,2022,1,1,0
1,0,2022-01-01 01:00:00,2022,1,1,1
2,0,2022-01-01 02:00:00,2022,1,1,2
3,0,2022-01-01 03:00:00,2022,1,1,3
4,0,2022-01-01 04:00:00,2022,1,1,4
...,...,...,...,...,...,...
38219875,999,2022-12-31 19:00:00,2022,12,31,19
38219876,999,2022-12-31 20:00:00,2022,12,31,20
38219877,999,2022-12-31 21:00:00,2022,12,31,21
38219878,999,2022-12-31 22:00:00,2022,12,31,22


In [8]:
# Merge the all_combinations dataframe with the hourly_sales_sample dataframe
merged_df = pd.merge(all_combinations, hourly_sales_sample, on=['product_id', 'year', 'month', 'day', 'hour'], how='left')

# Fill missing values in the hourly_sales column with 0
merged_df['hourly_sales'] = merged_df['hourly_sales'].fillna(0)

In [ ]:
merged_df['day_of_week'] = merged_df['timestamp'].dt.dayofweek

In [9]:
def get_instock_fraction_temp(product_id):
    df_stockout = stockout_all_products[stockout_all_products['product_id'] == product_id]
    if len(df_stockout)!=0:
        if df_stockout.iloc[0].stock_out_timestamps!='[]':
            temp_value = df_stockout.iloc[0].stock_out_timestamps
            # Extract timestamp strings from the string
            timestamp_strings = re.findall(r"\('(.*?)'\)", temp_value)
            # Remove the timezone information from the timestamp strings
            timestamp_strings = [t.split('+')[0] for t in timestamp_strings]
            # Convert timestamp strings to datetime objects
            datetime_list = [datetime.strptime(t, "%Y-%m-%d %H:%M:%S") for t in timestamp_strings]
            # Create list of tuples with pairs of datetime objects
            tuple_list = [(datetime_list[i], datetime_list[i+1]) for i in range(0, len(datetime_list), 2)]

            for index, row in merged_df.iterrows():
                # Extract year, month, day, and hour from the row

                year = int(row['year'])
                month = int(row['month'])
                day = int(row['day'])
                hour = int(row['hour'])
                # Create start time and end time by combining date and hour
                #start_datetime = pd.to_datetime(f'{year}-{month:02d}-{day:02d} {hour:02d}:00:00')
                #end_datetime = pd.to_datetime(f'{year}-{month:02d}-{day:02d} {hour:02d}:59:59')
                start_datetime = datetime(year, month, day, hour)
                end_datetime = start_datetime + timedelta(hours=1)

                # Initialize instock_fraction and overlap_duration
                instock_fraction = 1
                overlap_duration = timedelta()

                # Iterate over each tuple in tuple_list
                for start_end in tuple_list:
                    start = start_end[0]
                    end = start_end[1]

                    # Check if there is an overlap between the current hour and the tuple
                    if start <= end_datetime and end >= start_datetime:
                        # Calculate the overlap duration
                        overlap_start = max(start, start_datetime)
                        overlap_end = min(end, end_datetime)
                        overlap_duration += overlap_end - overlap_start + timedelta(seconds=1)

                # Check if there was any overlap
                if overlap_duration.total_seconds() > 0:
                    # Calculate instock_fraction as the fraction of overlap duration over one hour
                    instock_fraction = 1 - overlap_duration.total_seconds() / 3600

                # Mark the instock_fraction in the corresponding row of hourly_sales
                merged_df.at[index, 'instock_fraction'] = instock_fraction

In [ ]:
get_instock_fraction_temp('3834')

In [ ]:
merged_df['stockout_dum'] = (1 - merged_df['instock_fraction']).gt(1/60).astype(int)

In [ ]:
df_cooccurrence_of3834 = pd.read_csv("final_co_occurrence_ratio_for_3834.csv")

In [ ]:
df_sorted = df_cooccurrence_of3834.sort_values('co_occurrence_ratio_wrt_product_1', ascending=False)

top = 11
# Get the 40 biggest 'idpx' values
top_biggest = df_sorted[1:top]['product_id_2']

top_biggest

In [ ]:
merged_df['com_dum']=0
merged_df.loc[merged_df['product_id'].isin(top_biggest),'com_dum']=1

merged_df['stockout_com_interaction'] = merged_df['stockout_dum']*merged_df['com_dum']
merged_df